In [ ]:
!pip install tensorflow==2.4.0
!pip install keras==2.4.0
!pip install kymatio
!pip install tqdm
!pip install iterative-stratification
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.7 MB 17 kB/s 
     |████████████████████████████████| 462 kB 69.6 MB/s 
     |████████████████████████████████| 14.8 MB 51.6 MB/s 
     |████████████████████████████████| 132 kB 56.9 MB/s 
     |████████████████████████████████| 3.8 MB 40.5 MB/s 
     |████████████████████████████████| 2.9 MB 47.2 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68716 sha256=b9434c9ea435f9e8f5ae57e97394e0c55c3a50b25237ff750d1365f9efdccbd6
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: numpy
    Found existing installation: n

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 170 kB 5.1 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 3.8 MB/s 


In [ ]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
from absl import app, flags
from easydict import EasyDict
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import pickle
import sys
sys.path.append("drive/MyDrive/Scattering_Novo/src")
from DataHandler import DataHandler
from ModelHandler import ModelHandler
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

 
configs = {
    "N_GRIDS": 5, 
    "SIGNAL_BASE_LENGTH": 12800, 
    "N_CLASS": 26, 
    "USE_NO_LOAD": False, 
    "USE_HAND_AUGMENTATION": False,
    "MARGIN_RATIO": 0.15, 
    "DATASET_PATH": "drive/MyDrive/YOLO_NILM/Synthetic_Full_iHall.hdf5",
    "TRAIN_SIZE": 0.9,
    "FOLDER_PATH": "drive/MyDrive/Scattering_Novo/tmp/DIFDUAL/tests/AND/AND100_4/", 
    "FOLDER_DATA_PATH": "drive/MyDrive/Scattering_Novo/tmp/Without_Detection_Without_HAND/ND100/", 
    "N_EPOCHS_TRAINING": 5000,
    "PERCENTUAL": [1],
    "INITIAL_EPOCH": 0,
    "TOTAL_MAX_EPOCHS": 5000,
    "SNRdb": None # Nível de ruído em db
}

def freeze(model, task_name='classification'):
    for layer in model.layers:
        if task_name in layer.name:
            layer.trainable = True
        else:
            layer.trainable = False

    for layer in model.layers:
        print(layer.name, layer.trainable)

    return model

def calculating_class_weights(y_true):
    '''
        Source: https://stackoverflow.com/questions/48485870/multi-label-classification-with-class-weights-in-keras
    '''
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight(class_weight='balanced', classes=[0.,1.], y=y_true[:, i])
    return weights


def reduce_dataset(X_all,ydet_all,ytype_all,yclass_all,percentual):
    import numpy as np
    max_index = int(percentual*X_all.shape[0])
    np.random.seed(100)
    index = np.random.randint(max_index,size=(max_index-1))
    X_all = X_all[index]
    ydet_all = ydet_all[index]
    ytype_all = ytype_all[index]
    yclass_all = yclass_all[index]

    return X_all,ydet_all,ytype_all,yclass_all

ngrids = configs["N_GRIDS"]
signalBaseLength = configs["SIGNAL_BASE_LENGTH"]
trainSize = configs["TRAIN_SIZE"]
folderDataPath = configs["FOLDER_DATA_PATH"]
 
dataHandler = DataHandler(configs)


if not os.path.isfile(folderDataPath + "data.p"):
    print("Sorted data not found, creating new file...")
    x, ydet, yclass, ytype, ygroup = dataHandler.loadData(hand_augmentation=configs["USE_HAND_AUGMENTATION"], SNR=configs["SNRdb"])
    print("Data loaded")

    data_mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    strat_classes = np.max(yclass, axis=1)
    train_index, test_index = next(data_mskf.split(x, strat_classes))

    y_train = {
        "detection": ydet[train_index], 
        "type": ytype[train_index], 
        "classification": yclass[train_index], 
        "group": ygroup[train_index]
    }
    
    y_test = {
        "detection": ydet[test_index], 
        "type": ytype[test_index], 
        "classification": yclass[test_index], 
        "group": ygroup[test_index]
    }
    
    dict_data = {
        "x_train": x[train_index], 
        "x_test": x[test_index], 
        "y_train": y_train, 
        "y_test": y_test
    }

    print("Data sorted")

    try:
        os.mkdir(folderDataPath)
    except:
        pass

    pickle.dump(dict_data, open(folderDataPath + "data.p", "wb"))
    print("Data stored")
else:
    dict_data = pickle.load(open(folderDataPath + "data.p", "rb"))



modelHandler = ModelHandler(configs)
 
X_all = dict_data["x_train"]
ydet_all = dict_data["y_train"]["detection"]
ytype_all = dict_data["y_train"]["type"]
yclass_all = dict_data["y_train"]["classification"]

if configs["PERCENTUAL"][0]!=1:
        X_all,ydet_all,ytype_all,yclass_all = reduce_dataset(X_all,ydet_all,ytype_all,yclass_all,configs["PERCENTUAL"][0])

print(X_all.shape)
print(dict_data["x_test"].shape)


Using TensorFlow backend.


(7575, 16640, 1)
(841, 16640, 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, GlobalAveragePooling1D, Flatten, MaxPool1D, GlobalMaxPooling1D, BatchNormalization
from tensorflow.keras.models import Model
from kymatio.keras import Scattering1D

def buildBaseScattering(input_shape):
    '''
        Source: https://github.com/kymatio/kymatio/blob/master/examples/1d/classif_keras.py
    '''
    log_eps = 1e-6
    
    input = Input(shape=(input_shape,))
    x = Scattering1D(10, 10, max_order=1)(input) 

    unmapped_len = int(0.15 * (x.shape[2] / 1.3))
    grid_len = int((x.shape[2] - 2 * unmapped_len) / 5)

    print(f"X: {x.shape[2]}, Unmapped: {unmapped_len}, Grid: {grid_len}")

    left = Lambda(lambda x: x[..., :, : unmapped_len], name='left')(x)
    center = Lambda(lambda x: x[..., :, unmapped_len : x.shape[2] - unmapped_len], name='center')(x)
    right = Lambda(lambda x: x[..., :, x.shape[2] - unmapped_len :], name='right')(x)

    g1 = Lambda(lambda x: x[..., :, :grid_len], name='g1')(center)
    g2 = Lambda(lambda x: x[..., :, grid_len:2*grid_len], name='g2')(center)
    g3 = Lambda(lambda x: x[..., :, 2*grid_len:3*grid_len], name='g3')(center)
    g4 = Lambda(lambda x: x[..., :, 3*grid_len:4*grid_len], name='g4')(center)
    g5 = Lambda(lambda x: x[..., :, 4*grid_len:], name='g5')(center)


    leftav = tf.keras.backend.max(left, axis=2)
    g1av   = tf.keras.backend.max(g1, axis=2)
    g2av   = tf.keras.backend.max(g2, axis=2)
    g3av   = tf.keras.backend.max(g3, axis=2)
    g4av   = tf.keras.backend.max(g4, axis=2)
    g5av   = tf.keras.backend.max(g5, axis=2)
    rightav = tf.keras.backend.max(right, axis=2)
    
    
    x_type = tf.concat([(g1av-leftav), (g2av-leftav), (g3av-g1av), (g4av-g2av), (g5av-g3av), (rightav-g4av), (rightav-g5av) ], axis=1)
    x_class = tf.concat([leftav, g1av, g2av, g3av, g4av, g5av, rightav], axis=1)


    x_type = Flatten()(x_type)
    x_class = Flatten()(x_class)

    model = Model(inputs = input, outputs=[x_type, x_class])

    return model

tf.keras.backend.clear_session()

scattering_extract = buildBaseScattering(X_all.shape[1])
scattering_extract.summary()

X: 17, Unmapped: 1, Grid: 3
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 16640)]      0                                            
__________________________________________________________________________________________________
scattering1d (Scattering1D)     (None, 86, 17)       0           input_1[0][0]                    
__________________________________________________________________________________________________
center (Lambda)                 (None, 86, 15)       0           scattering1d[0][0]               
__________________________________________________________________________________________________
g1 (Lambda)                     (None, 86, 3)        0           center[0][0]                     
__________________________________________________________________

In [ ]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2_as_graph


def get_flops(model, batch_size=None):
    if batch_size is None:
        batch_size = 1

    real_model = tf.function(model).get_concrete_function(tf.TensorSpec([batch_size] + model.inputs[0].shape[1:], model.inputs[0].dtype))
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(real_model)

    run_meta = tf.compat.v1.RunMetadata()
    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
    flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                            run_meta=run_meta, cmd='op', options=opts)
    return flops.total_float_ops

def get_flops_output(model, batch_size=None):
    if batch_size is None:
        batch_size = 1
    argument = (tf.TensorSpec([batch_size] + model.inputs[0].shape[1:], model.inputs[0].dtype, name="type_input"),    
            tf.TensorSpec([batch_size] + model.inputs[1].shape[1:], model.inputs[0].dtype, name="class_input"))   
    real_model = tf.function(model).get_concrete_function(argument)
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(real_model)

    run_meta = tf.compat.v1.RunMetadata()
    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
    flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                            run_meta=run_meta, cmd='op', options=opts)
    return flops.total_float_ops
    

In [ ]:
flops_scatt = get_flops(scattering_extract, 32)


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


In [ ]:
print(f"FLOPS of Scattering Network: {flops_scatt / 10**9:.03} G")

FLOPS of Scattering Network: 0.226 G


In [ ]:
argument = (tf.TensorSpec([batch_size] + model.inputs[0].shape[1:], model.inputs[0].dtype, name="type_input"),    
            tf.TensorSpec([batch_size] + model.inputs[1].shape[1:], model.inputs[0].dtype, name="class_input"))


In [ ]:
real_model = tf.function(model).get_concrete_function(argument)
frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(real_model)

In [ ]:
run_meta = tf.compat.v1.RunMetadata()
opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                            run_meta=run_meta, cmd='op', options=opts)


In [ ]:
print(flops.total_float_ops)

37463840


Lets include the output network:

In [ ]:
fold = 1
folderPath = configs["FOLDER_PATH"] + str(fold) + "/"

model_output = ModelHandler.loadModel(folderPath + 'model_without_detection.h5')

In [ ]:
flops_output = get_flops_output(model_output, 32)

print(f"Scattering FLOPS: {flops_scatt / 10**9:.03} G")


print(f"Output FLOPS: {flops_output / 10**9:.03} G")


total_flops = flops_scatt + flops_output
print(f"Total FLOPS: {total_flops/ 10**9:.03} G")



Scattering FLOPS: 0.226 G
Output FLOPS: 0.0375 G
Total FLOPS: 0.264 G


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 602)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 602)]        0                                            
__________________________________________________________________________________________________
type_dense_0 (Dense)            (None, 300)          180900      input_3[0][0]                    
__________________________________________________________________________________________________
classification_dense_0 (Dense)  (None, 300)          180900      input_2[0][0]                    
____________________________________________________________________________________________